In [2]:
from Data_Preprocessing import exp_var, hate_crime_combined, hc_by_year, region_grouped, race_grouped

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('once')

## NEXT for Supervised Learning

Aggregate based on bias type, show the number of incidents per year per bias type along with the president's political affiliation.  Use top 3-5 biases.

Then train test split and try to predict the political affiliation for a given year.

In [3]:
hate_crime

,DATA_YEAR,STATE_ABBR,REGION_NAME,POPULATION_GROUP_CODE,INCIDENT_DATE,TOTAL_OFFENDER_COUNT,OFFENDER_RACE,VICTIM_COUNT,OFFENSE_NAME,LOCATION_NAME,BIAS_DESC,VICTIM_TYPES,TOTAL_INCIDENTS
0,1991,AR,South,5,1991-08-31,1,White,1,Intimidation,Highway/Road/Alley/Street/Sidewalk,Anti-Black or African American,Individual,1
1,1991,AR,South,6,1991-09-19,1,Black or African American,1,Simple Assault,Highway/Road/Alley/Street/Sidewalk,Anti-White,Individual,1
2,1991,AR,South,3,1991-07-04,1,Black or African American,1,Aggravated Assault,Residence/Home,Anti-Black or African American,Individual,1
3,1991,AR,South,3,1991-12-24,1,Black or African American,2,Aggravated Assault;Destruction/Damage/Vandalis...,Highway/Road/Alley/Street/Sidewalk,Anti-White,Individual,1
4,1991,AR,South,3,1991-12-23,1,Black or African American,1,Aggravated Assault,Service/Gas Station,Anti-White,Individual,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
219572,2020,WY,West,5,2020-11-29,6,White,2,Intimidation,Park/Playground,Anti-Lesbian (Female),Individual,1
219573,2020,WY,West,5,2020-07-14,0,Unknown,1,Destruction/Damage/Vandalism of Property,Park/Playground,Anti-White,Government,1
219574,2020,WY,West,5,2020-10-16,1,Black or African American,1,Aggravated Assault,Residence/Home,Anti-White,Individual,1
219575,2020,WY,West,5,2020-01-30,1,White,1,Simple Assault,Residence/Home,Anti-Female,Individual,1


In [9]:
#group by bias type -- ADD THIS TO VSCODE LATER

bias_grouped = hate_crime.groupby(['DATA_YEAR','BIAS_DESC'])['TOTAL_INCIDENTS'].sum().reset_index()
bias_grouped

,DATA_YEAR,BIAS_DESC,TOTAL_INCIDENTS
0,1991,Anti-Arab,73
1,1991,Anti-Asian,269
2,1991,Anti-Atheism/Agnosticism,4
3,1991,Anti-Bisexual,1
4,1991,Anti-Black or African American,1629
...,...,...,...
688,2020,Anti-Physical Disability,72
689,2020,Anti-Protestant,40
690,2020,Anti-Sikh,172
691,2020,Anti-Transgender,225


In [15]:
#plot